# ONE API modes
## Online vs Offline
ONE can be instantiated in either the online and offline state.  When online, ONE can query the
Alyx database to retrieve information about dataset types <TODO link>, search with metadata such as
session QC, task performance, histology and session narrative.  Advanced queries may also be
made [via REST](../one_advanced/one_advanced.html).  Other online methods include `pid2eid` and
`describe_revision`.

When the mode is not specified, it is usually set to 'auto', unless a cache_dir is specified for
which no database has been configured.

In [2]:
from one.api import ONE

one = ONE()
print(one, one.mode)  # online, 'auto' mode
assert not one.offline

one_offline = ONE(cache_dir=r'C:\data')
print(one_offline, one_offline.mode)  # offline, 'local' mode
assert one_offline.offline

One (online, https://alyx.internationalbrainlab.org) auto
One (offline, C:\data) local


## Query modes
In 'auto' mode, the list, search and load methods will use the local cache tables and not
connect to Alyx, however the option to use Alyx is always there.  If the cache tables can't be
downloaded from Alyx, or authentication fails, the mode will fall back to 'local'.

If 'remote' mode is specified, ONE will only query the remote database and will not use the
local cache tables.  Avoiding the database whenever possible is recommended as it doesn't rely
on a stable internet connection and reduces the load on the remote Alyx.

While in 'remote' mode, the local cache may be used by providing the query_type='local' keyword
argument to any method.  Likewise, in 'auto'/'local' mode, a remote query can be made by
specifying `query_type='remote'`

<div class="alert alert-info">
NB: The 'remote' query type is not valid in offline mode as there is no database associated to
the local cache directory.
</div>

In [3]:
eids = one.search(lab='cortexlab', query_type='remote')  # Search Alyx instead of the local cache

## Refreshing the cache
By default ONE will try to update the cache once every 24 hours.  This can be set by changing
the 'cache_expiry' property:

In [4]:
from datetime import timedelta
one.cache_expiry = timedelta(hours=1)  # Check for new remote cache every hour

# The time when the cache was generated can be found in the cache metadata:
one._cache._meta

{'expired': False,
 'created_time': datetime.datetime(2021, 9, 14, 13, 0),
 'loaded_time': datetime.datetime(2021, 9, 14, 18, 15, 54, 384591),
 'raw': {'datasets': {'date_created': '2021-09-14 13:00', 'origin': 'alyx'},
  'sessions': {'date_created': '2021-09-14 13:00', 'origin': 'alyx'}}}

Note that the cache won't be downloaded if the remote cache hasn't been updated since the last
download.  The cache can be explicitly refreshed in two ways:

In [5]:
loaded_time = one.refresh_cache('refresh')  # Explicitly refresh the cache
eids = one.search(lab='cortexlab', query_type='refresh')  # Calls `refresh_cache` before searching